In [1]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
library(optparse)
options(warn=-1)
options(scipen=999)
#library(crosstable)
options(error = quote({
  dump.frames(to.file=T, dumpto='last.dump')
  load('last.dump.rda')
  print(last.dump)
  q()
}))

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘tibble’ was built under R version 4.2.3”
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘caret’ was built under R version 4.2.3”
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked fro

In [12]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100,500,1000)
time_off<-c(10000,20,50,100)


lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)


In [13]:
m<-1
v<-1
#list_of_vi <- vector(mode="list", length=5)
#names(list_of_vi) <- c('10000', '10', '20', '50', '100')

list_of_vi_10000<-list()
list_of_vi_10<-list()
list_of_vi_20<-list()
list_of_vi_50<-list()
list_of_vi_100<-list()
list_of_rsqr=list()

corpus_list<-c("features")

ratings_list<-c("reddy","cordeiro90","cordeiro100")#,"reddy_true","cordeiro90_true","cordeiro100_true")
tagged_list<-c("Tagged")

ppmi_setting_list<-c("PPMI")
comp_setting_list<-c("Agnostic")
impute_list<-c("med")
to_predict_list<-c("compound","modifier","head")

feature_setting_list<-c("all")#,"cordeiro","cosine_sim","with_setting","info_theory","freq","prod")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

needed_cols<-c('modifier','head','avgModifier','stdevModifier','avgHead','stdevHead','compositionality','stdevHeadModifier','is_adj','compound','source')

cordeiro_cols<-c('arith_mean_sim.','beta.','geom_mean_sim.','sim_cpf_0.','sim_cpf_100.','sim_cpf_25.','sim_cpf_50.','sim_cpf_75.','sim_cpf_beta.')
cosine_sim_cols<-c("sim_bw_constituents.",'sim_with_head.','sim_with_modifier.')

with_setting_cols<-c("sim_bw_settings_comp.","sim_bw_settings_head.","sim_bw_settings_modifier.")
info_theory_cols<-c("local_mi.","log_ratio.","ppmi.")
freq_cols<-c("comp_freq.","comp_tf.","head_freq.","head_tf.","log_comp_freq.","log_head_freq.","log_head_freq_new.","log_mod_freq.","log_mod_freq_new.","mod_freq.","mod_tf.")
prod_cols<-c("head_family_size.","head_family_size_new.","head_prod.","mod_family_size.","mod_family_size_new.","mod_prod.")


In [14]:
dim(df_features_X)

[1]  90 897

In [19]:
for (c in corpus_list){
  for (t in tagged_list) {
    
    for (p in ppmi_setting_list){
      for (a in comp_setting_list){
        for (i in time_off) {
          
          for (j in cut_off) {
            
            for (im in impute_list) {
              
              #print(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_withSetting_",p,"_",t,"_",i,"_",j,"_",im,".csv")) 
              input_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_withSetting_",p,"_",t,"_",i,"_",j,"_",im,".csv"),sep = '\t')
              input_df <- input_df %>% distinct()
              if (a=="Aware" & i!=10000) {
                temp_features_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/temporal_Compound",a,"_withSetting_",p,"_",t,"_",i,"_",j,"_",im,".csv"),sep = '\t')
                temp_features_df<-temp_features_df %>% distinct()
                input_df<-merge(input_df,temp_features_df,all.x=TRUE)
                input_df <- input_df %>% distinct()
              }
              
              for (r in ratings_list){
                
                reduced_df<-input_df %>% filter(source==r)
                reduced_df <- reduced_df %>% filter(!compound %in% c('call_centre','benign_tumour', 'computer_programme', 'grey_matter', 'labour_union','baby_blues', 'contact_lenses'))
                
                
                
                
                if (t=="Tagged") {
                  compounds_list<-c("an","nn")
                }
                else {
                  compounds_list<-c("all")
                  
                }
                for (k in compounds_list) {
                  df_features<-reduced_df
                  
                  if (k=="an") { 
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_NOUN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    df_features<-rbind(group1_df,group2_df)                   
                  }
                  else if (k=="nn") {
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    df_features<-rbind(group1_df,group2_df)                    
                  }
                  
                  else if (k=="ap") {
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_PROPN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_PROPN",modifier) & grepl("_PROPN",head))
                    df_features<-rbind(group1_df,group2_df)                    
                  }
                  
                  else if (k=="pp") {
                    group1_df<-df_features %>% filter(is_adj=="True" & grepl("_PROPN",modifier) & grepl("_PROPN",head))
                    group2_df<-df_features %>% filter(is_adj=="False" & grepl("_PROPN",modifier) & grepl("_PROPN",head))
                    df_features<-rbind(group1_df,group2_df)                    
                  }
                  
                  df_features_X<-df_features %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                  for (f in feature_setting_list) {
                    
                    if(f=="cordeiro"){
                      trainX<-df_features_X %>% select(starts_with(cordeiro_cols))
                    }
                    else if(f=="cosine_sim"){
                      trainX<-df_features_X %>% select(starts_with(cosine_sim_cols))
                      
                    }
                    
                    else if(f=="with_setting"){
                      trainX<-df_features_X %>% select(starts_with(with_setting_cols))
                    }
                    
                    else if(f=="info_theory"){
                      trainX<-df_features_X %>% select(starts_with(info_theory_cols))
                    }
                    
                    else if(f=="freq"){
                      trainX<-df_features_X %>% select(starts_with(freq_cols))
                    }
                    else if(f=="prod"){
                      trainX<-df_features_X %>% select(starts_with(prod_cols))
                    }
                    else {
                      trainX<-df_features_X
                    }
                    
                    if (dim(trainX)[1]<10) {
                      break
                    }
                    for (pr in to_predict_list){
                      
                      if (pr=="compound") {
                        trainY<-df_features %>% select(compositionality)
                        trainY<-trainY$compositionality      
                      }
                      
                      else if (pr=="modifier") {
                        trainY<-df_features %>% select(avgModifier)
                        trainY<-trainY$avgModifier      
                      }
                      
                      else if (pr=="head") {
                        trainY<-df_features %>% select(avgHead)
                        trainY<-trainY$avgHead      
                      }                               
                      
                      print(paste0(c," ",t," ",p," ",a," ",i," ",j," ",im," ",r," ",k," ",f," ",pr))
                      
                      print(dim(trainX))
                      
                      for (s in seed_list)  {
                        set.seed(s)
                        seeds <- vector(mode = "list", length = 14)
                        for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                        #for the last model
                        seeds[[14]]<-sample.int(1000, 1)
                        
                        preprocess_list<-c("nzv","medianImpute", "center", "scale")
                        
                        
                        if (sum(is.na(trainX))>1 & im=="med") {
                          print("NAs found")}                         
                        
                        elastic_model <- train(trainX,trainY,method = "glmnet",metric = "Rsquared",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                               preProcess = preprocess_list)
                        
                        elastic_spearman_model <- train(trainX,trainY,method = "glmnet",metric = "Spearman",
                                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                        preProcess = preprocess_list)
                        
                        print(dim(trainX))
                        perf_elastic<-data.frame(corpus=c,tag=t,ppmi=p,setting=a,timespan=i,cutoff=j,impute=im,dataset=r,pattern=k,features=f,y=pr,n=nrow(trainX),seed=s,ml_algo="elastic",method=getTrainPerf(elastic_model)[,"method"],TrainRsquared=getTrainPerf(elastic_model)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_spearman_model)[,"TrainSpearman"])
                        
                        varimp_elastic<-data.frame(corpus=c,tag=t,ppmi=p,setting=a,timespan=i,cutoff=j,impute=im,dataset=r,pattern=k,features=f,y=pr,n=nrow(trainX),seed=s,ml_algo="elastic",t(varImp(elastic_model)$importance))
                        
                        
                        list_of_rsqr[[m]]<-perf_elastic
                        m<-m+1 
                        
                        if (i==10000) {
                          list_of_vi_10000[[v]]<-varimp_elastic
                          v<-v+1
                        }
                        else if (i==10) {
                          list_of_vi_10[[v]]<-varimp_elastic
                          v<-v+1
                        }
                        
                        else if (i==20) {
                          list_of_vi_20[[v]]<-varimp_elastic
                          v<-v+1
                        }
                        else if (i==50) {
                          list_of_vi_50[[v]]<-varimp_elastic
                          v<-v+1
                        }
                        else if (i==100) {
                          list_of_vi_100[[v]]<-varimp_elastic
                          v<-v+1
                        }                                                                                                   
                        
                      }
                      rsquared_df<-bind_rows(list_of_rsqr)
                      rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
                      print(dim(rsquared_df))
                      
                      #write.csv(rsquared_df,paste0("~/rsquared_",c,"_",t,"_",p,"_",a,"_",i,"_",j,"_",im,"_",r,"_",k,"_",f,"_",pr,".csv"),row.names = FALSE)
                      list_of_rsqr<-list()
                      if (i==10000) {
                        varimp_10000_df<-bind_rows(list_of_vi_10000)
                        varimp_10000_df$cutoff<-as.factor(varimp_10000_df$cutoff)
                        varimp_10000_df[is.na(varimp_10000_df)] <- 0
                        #write.csv(varimp_10000_df,paste0("~/varimp_10000_",c,"_",t,"_",p,"_",a,"_",i,"_",j,"_",im,"_",r,"_",k,"_",f,"_",pr,".csv"),row.names = FALSE)
                        list_of_vi_10000<-list()
                        
                      }
                      else if (i==10) {
                        varimp_10_df<-bind_rows(list_of_vi_10)
                        varimp_10_df$cutoff<-as.factor(varimp_10_df$cutoff)
                        varimp_10_df[is.na(varimp_10_df)] <- 0
                        # write.csv(varimp_10_df,paste0("~/varimp_10_",c,"_",t,"_",p,"_",a,"_",i,"_",j,"_",im,"_",r,"_",k,"_",f,"_",pr,".csv"),row.names = FALSE)
                        list_of_vi_10<-list()
                      }
                      
                      else if (i==20) {
                        varimp_20_df<-bind_rows(list_of_vi_20)
                        varimp_20_df$cutoff<-as.factor(varimp_20_df$cutoff)
                        varimp_20_df[is.na(varimp_20_df)] <- 0
                        # write.csv(varimp_20_df,paste0("~/varimp_20_",c,"_",t,"_",p,"_",a,"_",i,"_",j,"_",im,"_",r,"_",k,"_",f,"_",pr,".csv"),row.names = FALSE)
                        list_of_vi_20<-list()
                        
                      }
                      else if (i==50) {
                        varimp_50_df<-bind_rows(list_of_vi_50)
                        varimp_50_df$cutoff<-as.factor(varimp_50_df$cutoff)
                        varimp_50_df[is.na(varimp_50_df)] <- 0
                        #write.csv(varimp_50_df,paste0("~/varimp_50_",c,"_",t,"_",p,"_",a,"_",i,"_",j,"_",im,"_",r,"_",k,"_",f,"_",pr,".csv"),row.names = FALSE)
                        list_of_vi_50<-list()
                        
                      }
                      else if (i==100) {
                        varimp_100_df<-bind_rows(list_of_vi_100)
                        varimp_100_df$cutoff<-as.factor(list_of_vi_100$cutoff)
                        varimp_100_df[is.na(list_of_vi_100)] <- 0
                        #write.csv(varimp_100_df,paste0("~/varimp_50_",c,"_",t,"_",p,"_",a,"_",i,"_",j,"_",im,"_",r,"_",k,"_",f,"_",pr,".csv"),row.names = FALSE)
                        list_of_vi_100<-list()
                        
                      } 
                      
                      
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}

[1] "features Tagged PPMI Agnostic 10000 0 med reddy an all compound"
[1] 90 33


In [18]:
group2_df %>% distinct()

modifier,head,arith_mean_sim.0,beta.0,comp_freq.0,comp_size.0,comp_tf.0,geom_mean_sim.0,head_family_size.0,head_family_size_new.0,⋯,sim_with_modifier.0,avgModifier,stdevModifier,avgHead,stdevHead,compositionality,stdevHeadModifier,is_adj,compound,source
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
end_NOUN,user_NOUN,0.21266660,0.06212472,2108.41534,165456237,11.042627,0.10836946,-11.964496,-10.999458,⋯,0.02968260,3.866667,1.117537,4.866667,0.339935,4.250000,0.871165,False,end_user,reddy
firing_NOUN,line_NOUN,0.11942735,0.69566932,1528.46273,165456237,10.578809,0.11054376,-9.156460,-8.191421,⋯,0.16462657,1.607143,1.654848,1.892857,1.496169,1.703704,1.717337,False,firing_line,reddy
game_NOUN,plan_NOUN,0.09923754,0.33646010,738.99377,165456237,9.531369,0.09392202,-10.056730,-9.091692,⋯,0.06719469,2.821429,1.964935,4.862069,0.344828,3.827586,1.233693,False,game_plan,reddy
application_NOUN,form_NOUN,0.15646479,0.67048782,1457.38643,165456237,10.510157,0.14804607,-8.845564,-7.880526,⋯,0.20709673,4.766667,0.422953,4.862069,0.344828,4.800000,0.476095,False,application_form,reddy
snail_NOUN,mail_NOUN,0.07899969,0.31629982,238.22299,165456237,7.902212,0.07360390,-12.977694,-12.012655,⋯,0.05030444,0.600000,0.800000,4.586207,1.099129,1.310345,1.020596,False,snail_mail,reddy
web_NOUN,site_NOUN,0.32932308,0.74104931,2546.22884,165456237,11.314713,0.30329867,-10.725523,-9.760485,⋯,0.45763369,2.678571,1.691440,3.933333,1.181336,3.785714,1.205853,False,web_site,reddy
flea_NOUN,market_NOUN,0.15905700,0.70514365,1071.71793,165456237,10.067055,0.14629798,-10.238385,-9.273346,⋯,0.22147519,0.379310,0.805746,4.714286,0.839096,1.517241,1.133219,False,flea_market,reddy
grandfather_NOUN,clock_NOUN,0.18984188,0.32179600,995.56398,165456237,9.960819,0.17931702,-13.623825,-12.658786,⋯,0.12750933,0.428571,0.775913,5.000000,0.000000,2.642857,1.315139,False,grandfather_clock,reddy
diamond_NOUN,wedding_NOUN,0.10361380,0.35800843,178.02149,165456237,7.483989,0.09951264,-15.008403,-14.043364,⋯,0.07475112,1.066667,1.289272,3.407407,1.340516,1.703704,1.047566,False,diamond_wedding,reddy


In [ ]:
return (1 - ((1-summary(x)$r.squared)*(nobs(x)-1)/(nobs(x)-length(x$coefficients)-1)))


In [59]:
TrainRsquared<-getTrainPerf(elastic_model)[,"TrainRsquared"]
TrainRsquared

[1] 0.5071782

In [62]:
1 - ((1-TrainRsquared)*(nobs-1)/(nobs-real_coef-1))

[1] 8.089053

In [57]:
nobs<-dim(trainX)[1]

In [53]:
real_coef<-length(predictors(elastic_model))

In [67]:
length(predictors(elastic_model))

[1] 102

In [33]:
sum_obj<-summary(elastic_model)
sum_obj[[3]]

[1] "  100"